# **1. Imagen de características especio temporales**

In [1]:
import cv2
import tensorflow as tf 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np

In [2]:
base_model=tf.keras.applications.VGG16(
                include_top=False,
                weights='imagenet',
                input_shape=(224,224,3)
            )

In [3]:
model=Sequential()
model.add(base_model)
model.add(Flatten())

In [4]:
def readVideo(path):
    frames=[]
    vidcap=cv2.VideoCapture(path)
    success,image=vidcap.read()
    
    while success:
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        frames.append(cv2.resize(image,(224,224)))
        success,image=vidcap.read()

    return frames

In [5]:
frames=readVideo("/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_100.mp4")

In [6]:
len(frames)

In [7]:
plt.imshow(frames[3])

In [8]:
frames[0].shape

In [9]:
def convFeaatureImage(frames):
    conv_features=model.predict(np.array(frames))
    return np.array(conv_features)

In [10]:
img_features = convFeaatureImage(frames)

In [11]:
img_features.shape

# **2. Generación del dataset de entrenamiento "inducción al error"**

In [12]:
import glob

In [13]:
paths=["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/",
      "/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/"]

In [14]:
def resize_zeros(img_features,maxFrames):
    rows,col=img_features.shape
    zeroMatrix=np.zeros((maxFrames-rows,col))
    return np.concatenate((img_features,zeroMatrix),axis=0)

In [15]:
resize_zeros(img_features,180).shape

In [16]:
from tqdm import tqdm

In [17]:
"""
data=[]
#sizes=[]
maxFrames=190
for label,path in enumerate(paths):
    for file in tqdm(glob.glob(path+'*.mp4')):
        frames=readVideo(file)
        #sizes.append(len(frames))
        if len(frames)<maxFrames:
            img_features=convFeaatureImage(frames)
            data.append([label,file,resize_zeros(img_features,maxFrames)])
"""
        

In [18]:
maxFrames=190

In [19]:
#plt.hist(sizes,1000,range=[0,200])

# **3. Escribir TFrecords**

In [20]:
import random 
random.seed(0)

In [21]:
label_videoPath=[]
for label,path in enumerate(paths):
    for file in glob.glob(path+'*.mp4'):
        if random.random()<=0.25:
            label_videoPath.append((label,file))

In [22]:
random.shuffle(label_videoPath)

In [23]:
label_videoPath[0]

In [24]:
eval_files=[file for label,file in label_videoPath[int(0.9*len(label_videoPath))+1::]]
test_files=[file for label,file in label_videoPath[int(0.8*len(label_videoPath))+1:int(0.9*len(label_videoPath))]]

In [25]:
test_files

In [26]:
print(len(eval_files),len(test_files))

In [27]:
def createTfExample(img_feature,label):
    sample={ 'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_features.shape[0]])),
             'width':  tf.train.Feature(int64_list=tf.train.Int64List(value=[img_features.shape[1]])),
             'image':  tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_features.tobytes()])),
             'label':  tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }
    
    return tf.train.Example(features=tf.train.Features(feature=sample))

In [28]:
train_writer=tf.io.TFRecordWriter('train.record')
eval_writer=tf.io.TFRecordWriter('eval.record')
test_writer=tf.io.TFRecordWriter('test.record')
for label, file in tqdm(label_videoPath):
    frames=readVideo(file)
    if len(frames)<maxFrames:
        img_features=convFeaatureImage(frames)
        img_features=resize_zeros(img_features,maxFrames)
        example=createTfExample(img_features,label)
        if file in eval_files:
            eval_writer.write(example.SerializeToString())
        elif file in test_files:
            test_writer.write(example.SerializeToString())
        else:
            train_writer.write(example.SerializeToString())
train_writer.close()
eval_writer.close()
test_writer.close()

# **4. Leer los TFrecords**

In [59]:
import tensorflow as tf

In [60]:
dataset_train=tf.data.TFRecordDataset("train.record")
dataset_eval=tf.data.TFRecordDataset("eval.record")
dataset_test=tf.data.TFRecordDataset("test.record")

In [61]:
feature_format={
    'height': tf.io.FixedLenFeature([],tf.int64),
    'width':  tf.io.FixedLenFeature([],tf.int64),
    'image':  tf.io.FixedLenFeature([],tf.string),
    'label':  tf.io.FixedLenFeature([],tf.int64)
}

In [62]:
def get_dataFromBinary(proto):
    f=tf.io.parse_single_example(proto,feature_format)
    height=tf.cast(f['height'],tf.int32)
    width=tf.cast(f['width'],tf.int32)
    img=tf.io.decode_raw(f['image'],tf.float64)
    img=tf.reshape(img,[height,width])
    label=f['label']
    return (img,label)

In [63]:
def proc_dataset(dataset):
    dataset=dataset.map(get_dataFromBinary)
    dataset=dataset.batch(5)
    return dataset

In [64]:
dataset_train=proc_dataset(dataset_train)
dataset_eval=proc_dataset(dataset_eval)
dataset_test=proc_dataset(dataset_test)

# **5. Definición del modelo**

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv1D,MaxPooling1D,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [72]:
def cnn_model():
    model = Sequential()
    model.add(Conv1D(filters=128,kernel_size=4,input_shape=(190,25088),activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64,kernel_size=4,activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32,kernel_size=4,activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [73]:
modelCnn=cnn_model()

In [74]:
modelCnn.summary()

In [75]:
checkpoint=ModelCheckpoint('checkpoint/model.{epoch:d}.h5',save_best_only=False, save_freq='epoch')

In [76]:
modelCnn.fit(dataset_train,
            epochs=15,
            batch_size=5,
            validation_data=dataset_eval,
            callbacks=[checkpoint])

In [92]:
modelCnn.load_weights("./checkpoint/model.9.h5")

# **6. Evaluación del modelo**

In [93]:
modelCnn.evaluate(dataset_eval)

In [94]:
modelCnn.evaluate(dataset_test)

# **7. Predicción para un video**

In [98]:
frames=readVideo('/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_515.mp4')
img_features=convFeaatureImage(frames)
img_features=resize_zeros(img_features,maxFrames)
img_features.shape

In [99]:
if modelCnn.predict(np.array([img_features]))[0]>=0.5:
    print("El video es un situacion de violencia")
else:
    print("El video no es violento")